<a href="https://colab.research.google.com/github/Armin-Abdollahi/WhiteTree-International-Academy/blob/main/CogniFit_Report_Download.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
!pip install PyPDF2 pandas openpyxl matplotlib seaborn scipy

In [24]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from PIL import Image
from matplotlib.backends.backend_pdf import PdfPages
from PyPDF2 import PdfReader, PdfWriter
from google.colab import files # For downloading in Colab
import scipy.stats as stats

# Apply a seaborn style for better aesthetics
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.family'] = 'sans-serif'
# Attempt to use a common font, fallback to default if not available
try:
    plt.rcParams['font.sans-serif'] = ['Arial'] # Or 'Calibri', 'Helvetica'
except:
    print("Arial font not found, using Matplotlib default.")

plt.rcParams['axes.titlesize'] = 14 # Title font size for subplots
plt.rcParams['axes.labelsize'] = 10 # Axis label font size
plt.rcParams['xtick.labelsize'] = 8
plt.rcParams['ytick.labelsize'] = 8
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['patch.edgecolor'] = 'black' # Add border to bar charts for definition


def plot_maker(df_input, output_pdf='report.pdf', name="Armin Abdollahi", age=25, date="2025-01-31"):
    front_cover_path = '/content/Front_Cover.pdf'
    back_cover_path = '/content/Back_Cover.pdf'
    logo1_path = '/content/Logo1.png'
    logo2_path = '/content/WhiteTree.png'

    primary_color = sns.color_palette("pastel")[0]
    secondary_color = sns.color_palette("pastel")[1]
    text_color = "#333333"

    plt.rcParams['text.color'] = text_color
    plt.rcParams['axes.labelcolor'] = text_color
    plt.rcParams['xtick.color'] = text_color
    plt.rcParams['ytick.color'] = text_color
    plt.rcParams['axes.titlecolor'] = text_color

    logo1_resized = None
    logo2_resized = None

    try:
        logo1_img_pil = Image.open(logo1_path)
        logo1_original_w, logo1_original_h = 1255, 552
        new_width_px_logo1 = 120
        new_height_px_logo1 = int(new_width_px_logo1 * logo1_original_h / logo1_original_w)
        logo1_resized = logo1_img_pil.resize((new_width_px_logo1, new_height_px_logo1))
        print(f"Logo1 '{logo1_path}' loaded and resized to ({new_width_px_logo1}, {new_height_px_logo1}).")
    except FileNotFoundError:
        print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print(f"!!! هشدار: فایل لوگوی اول '{logo1_path}' یافت نشد. لوگو نمایش داده نخواهد شد. !!!")
        print(f"!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    except Exception as e:
        print(f"Error processing Logo1 file '{logo1_path}': {e}. Logo1 will not be displayed.")

    try:
        logo2_img_pil = Image.open(logo2_path)
        logo2_resized = logo2_img_pil.resize((70, 70))
        print(f"Logo2 '{logo2_path}' loaded and resized.")
    except FileNotFoundError:
        print(f"Warning: Logo2 file '{logo2_path}' not found. Skipping.")
    except Exception as e:
        print(f"Error processing Logo2 file '{logo2_path}': {e}. Skipping.")

    pdf = PdfPages(output_pdf)

    y_pos_header_line = 0.96
    x_pos_header_anchor = 0.95
    y_main_title = 0.90
    tight_layout_rect_top = 0.88

    # ----------------------------------------------- Page 2 -----------------------------------------------
    fig_page2 = plt.figure(figsize=(8.27, 11.69))

    if logo1_resized:
        fig_page2.figimage(logo1_resized, 50, fig_page2.bbox.ymax - 40 - logo1_resized.height, zorder=3)

    header_string_p2 = f"Name: {name}     Age: {age}     Date of Evaluation: {date}"
    fig_page2.text(x_pos_header_anchor, y_pos_header_line, header_string_p2, ha='right', fontsize=10, color=text_color)
    fig_page2.text(0.5, y_main_title, "Cognitive Profile", fontsize=14, ha='center', weight='bold', color=text_color)

    labels_page1_cognitive = [
        'Visual Memory', 'Working Memory', 'Short-term Memory',
        'Focus Attention', 'Updating', 'Hand-Eye Coordination',
        'Spatial Perception', 'Reaction Time', 'Processing Speed',
        'Planning and Reasoning'
    ]
    values_page1 = np.zeros(len(labels_page1_cognitive))
    if not df_input.empty:
        try:
            extracted_values = [df_input[label].iloc[0] if label in df_input.columns else 0 for label in labels_page1_cognitive]
            values_page1 = np.array(extracted_values, dtype=float)
        except Exception as e:
            print(f"Error extracting data for Page 2: {e}. Using zeros.")
    else:
        print("Warning: Input DataFrame empty for Page 2. Using zeros.")

    ax1 = fig_page2.add_subplot(2, 2, 1)
    ax1.axis('off')
    # --------- تغییر ۱: نمایش Final Score به صورت درصد ---------
    table_data_p1 = [[label, f"{val*100:.1f}%"] for label, val in zip(labels_page1_cognitive, values_page1)]
    table_p1 = ax1.table(cellText=table_data_p1, colLabels=["Cognitive Ability", "Final Score"],
                         cellLoc='left', loc='center', colColours=[primary_color, primary_color])
    table_p1.auto_set_font_size(False)
    table_p1.set_fontsize(9)
    table_p1.scale(1, 1.8)
    for key, cell in table_p1.get_celld().items():
        cell.set_edgecolor('lightgrey')
        if key[0] == 0:
            cell.set_text_props(weight='bold', color='white')
        else:
            cell.set_text_props(color=text_color)
            if key[0] % 2 == 0:
                   cell.set_facecolor('#f0f0f0')

    ax2 = fig_page2.add_subplot(2, 2, 2, polar=True)
    ax2.set_facecolor('#f9f9f9')
    values_polar = values_page1 * 100
    if values_polar.size == 0:
        values_polar = np.zeros(len(labels_page1_cognitive))
    plot_values_polar = np.append(values_polar, values_polar[0] if values_polar.size > 0 else 0)
    angles = np.linspace(0, 2 * np.pi, len(labels_page1_cognitive), endpoint=False).tolist()
    angles += angles[:1]
    ax2.fill(angles, plot_values_polar, color=primary_color, alpha=0.5, zorder=2)
    ax2.plot(angles, plot_values_polar, color=primary_color, linewidth=2, zorder=3)
    band_colors = ['#ffcccb', '#ffebcc', '#fff9cc', '#d9ead3']
    ranges = [(0, 25), (25, 50), (50, 75), (75, 100)]
    for i in range(len(ranges)):
        ax2.fill_between(angles, ranges[i][0], ranges[i][1], color=band_colors[i], alpha=0.3, zorder=1)
    ax2.plot(angles, [50] * len(angles), color='grey', linestyle='--', linewidth=1, zorder=3)
    ax2.set_xticks(angles[:-1])
    ax2.set_xticklabels(labels_page1_cognitive, fontsize=7, color=text_color)
    ax2.set_yticks(np.arange(0, 101, 25))
    ax2.set_yticklabels([f"{i}%" for i in np.arange(0, 101, 25)], color='grey', fontsize=7)
    ax2.set_rlabel_position(22.5)
    ax2.grid(True, color='lightgrey', linestyle='--')
    ax2.set_title('Cognitive Abilities Profile', fontsize=11, weight='bold', pad=30, color=text_color)

    ax3 = fig_page2.add_subplot(2, 1, 2)
    y_pos_p1 = np.arange(len(labels_page1_cognitive))
    bars1 = ax3.bar(y_pos_p1, values_page1, align='center', color=primary_color, edgecolor='grey')
    ax3.set_xticks(y_pos_p1)
    ax3.set_xticklabels(labels_page1_cognitive, rotation=45, ha="right", color=text_color)
    for bar_item in bars1:
        yval = bar_item.get_height()
        ax3.text(bar_item.get_x() + bar_item.get_width()/2.0, yval + 0.01, f"{yval*100:.1f}%", ha='center', va='bottom', fontsize=7, color=text_color)
    ax3.set_ylabel('Normalized Score (0-1)', color=text_color)
    ax3.set_title('Performance Metrics Overview', fontsize=11, weight='bold', color=text_color)
    ax3.spines['top'].set_visible(False)
    ax3.spines['right'].set_visible(False)
    ax3.set_ylim(0, max(1.0, np.max(values_page1) * 1.15 if values_page1.size > 0 else 1.0))

    if logo2_resized:
        fig_page2.figimage(logo2_resized, fig_page2.bbox.xmax - 50 - logo2_resized.width, fig_page2.bbox.ymin + 30, zorder=3)

    fig_page2.tight_layout(rect=[0, 0.05, 1, tight_layout_rect_top])
    pdf.savefig(fig_page2)
    plt.close(fig_page2)

    # ----------------------------------------------- Page 3 -----------------------------------------------
    fig_page3 = plt.figure(figsize=(8.27, 11.69))

    if logo1_resized:
        fig_page3.figimage(logo1_resized, 50, fig_page3.bbox.ymax - 40 - logo1_resized.height, zorder=3)

    header_string_p3 = f"Name: {name}     Age: {age}     Date of Evaluation: {date}"
    fig_page3.text(x_pos_header_anchor, y_pos_header_line, header_string_p3, ha='right', fontsize=10, color=text_color)
    fig_page3.text(0.5, y_main_title, "Assessment Results", fontsize=14, ha='center', weight='bold', color=text_color)

    labels_page2_assessments = [
        'Psychomotor Vigilance Test', 'Eye-Hand Coordination Test',
        'Eye-Hand Coordination MUD', 'Maze Test', 'Digit Span Test', 'Visual Memory Test'
    ]
    descriptions_page2 = ['Measures sustained attention', 'Assesses coordination skills', 'Multi-target coordination',
                          'Spatial navigation ability', 'Short-term memory capacity', 'Visual recall ability']
    values_page2 = np.zeros(len(labels_page2_assessments))
    if not df_input.empty:
        try:
            extracted_values_p2 = [df_input[label].iloc[0] if label in df_input.columns else 0 for label in labels_page2_assessments]
            values_page2 = np.array(extracted_values_p2, dtype=float)
        except Exception as e:
            print(f"Error extracting data for Page 3: {e}. Using zeros.")
    else:
        print("Warning: Input DataFrame empty for Page 3. Using zeros.")

    ax4 = fig_page3.add_subplot(2, 1, 1)
    ax4.axis('off')
    table_data_p2 = [['Assessment Name', 'Score', 'Description']]
    for i in range(len(labels_page2_assessments)):
        score_val = values_page2[i] if i < len(values_page2) else 0
        desc_val = descriptions_page2[i] if i < len(descriptions_page2) else "N/A"
        # --------- تغییر ۲: نمایش Score به صورت درصد ---------
        table_data_p2.append([labels_page2_assessments[i], f"{score_val*100:.1f}%", desc_val])
    table_p2 = ax4.table(cellText=table_data_p2, colLabels=None, cellLoc='left', loc='center',
                         colWidths=[0.35, 0.15, 0.5])
    table_p2.auto_set_font_size(False)
    table_p2.set_fontsize(9)
    table_p2.scale(1, 1.8)
    for key, cell in table_p2.get_celld().items():
        cell.set_edgecolor('lightgrey')
        if key[0] == 0: # Header
            cell.set_text_props(weight='bold', color='white')
            cell.set_facecolor(secondary_color) # Header color
        else: # Data rows
            cell.set_text_props(color=text_color)
            if key[0] % 2 == 0: # Alternate row color
                   cell.set_facecolor('#f0f0f0')
        if key[1] == 2: # Description column
            cell.set_text_props(wrap=True)

    ax5 = fig_page3.add_subplot(2, 1, 2)
    y_pos_p2 = np.arange(len(labels_page2_assessments))
    bars2 = ax5.bar(y_pos_p2, values_page2, align='center', color=secondary_color, edgecolor='grey')
    ax5.set_xticks(y_pos_p2)
    ax5.set_xticklabels(labels_page2_assessments, rotation=45, ha="right", color=text_color)
    for bar_item in bars2:
        yval = bar_item.get_height()
        ax5.text(bar_item.get_x() + bar_item.get_width()/2.0, yval + 0.01, f"{yval*100:.1f}%", ha='center', va='bottom', fontsize=7, color=text_color)
    ax5.set_ylabel('Normalized Score (0-1)', color=text_color)
    ax5.set_title('Assessment Metrics Overview', fontsize=11, weight='bold', color=text_color)
    ax5.spines['top'].set_visible(False)
    ax5.spines['right'].set_visible(False)
    ax5.set_ylim(0, max(1.0, np.max(values_page2) * 1.15 if values_page2.size > 0 else 1.0))

    if logo2_resized:
        fig_page3.figimage(logo2_resized, fig_page3.bbox.xmax - 50 - logo2_resized.width, fig_page3.bbox.ymin + 30, zorder=3)

    fig_page3.tight_layout(rect=[0, 0.05, 1, tight_layout_rect_top])
    pdf.savefig(fig_page3)
    plt.close(fig_page3)

    # ----------------------------------------------- Page 4 (New: Bell Curve) ----------------------------------
    fig_page4 = plt.figure(figsize=(8.27, 11.69))

    if logo1_resized:
        fig_page4.figimage(logo1_resized, 50, fig_page4.bbox.ymax - 40 - logo1_resized.height, zorder=3)

    header_string_p4 = f"Name: {name}     Age: {age}     Date of Evaluation: {date}"
    fig_page4.text(x_pos_header_anchor, y_pos_header_line, header_string_p4, ha='right', fontsize=10, color=text_color)
    fig_page4.text(0.5, y_main_title, "Performance Distribution Model", fontsize=14, ha='center', weight='bold', color=text_color)

    ax_bell = fig_page4.add_axes([0.2, 0.3, 0.6, 0.45])

    mean_bc = 50
    std_dev_bc = 16
    x_bc = np.linspace(0, 100, 1000)
    y_bc = stats.norm.pdf(x_bc, mean_bc, std_dev_bc)
    ax_bell.plot(x_bc, y_bc, color='darkblue', linewidth=2)
    ax_bell.fill_between(x_bc, y_bc, where=(x_bc < 25), color='red', alpha=0.3, interpolate=True)
    ax_bell.fill_between(x_bc, y_bc, where=(x_bc >= 25) & (x_bc < 50), color='orange', alpha=0.3, interpolate=True)
    ax_bell.fill_between(x_bc, y_bc, where=(x_bc >= 50) & (x_bc < 75), color='yellow', alpha=0.3, interpolate=True)
    ax_bell.fill_between(x_bc, y_bc, where=(x_bc >= 75), color='green', alpha=0.3, interpolate=True)
    ax_bell.axvline(x=25, color='grey', linestyle='--', linewidth=1)
    ax_bell.axvline(x=50, color='grey', linestyle='--', linewidth=1)
    ax_bell.axvline(x=75, color='grey', linestyle='--', linewidth=1)
    ax_bell.set_title('Illustrative Normal Distribution of Scores (0-100 scale)', fontsize=11, weight='bold', color=text_color)
    ax_bell.set_xlabel('Standardized Score Value', fontsize=10, color=text_color)
    ax_bell.set_ylabel('Probability Density', fontsize=10, color=text_color)
    ax_bell.grid(True, linestyle='--', linewidth=0.5, alpha=0.7)
    ax_bell.set_facecolor('white')
    ax_bell.spines['top'].set_visible(False)
    ax_bell.spines['right'].set_visible(False)
    ax_bell.spines['left'].set_color('gray')
    ax_bell.spines['bottom'].set_color('gray')
    region_texts = ["Below Average", "Low Average", "High Average", "Above Average"]
    region_x_positions = [12.5, 37.5, 62.5, 87.5]
    max_y_for_text = np.max(y_bc) * 0.05
    for i, txt in enumerate(region_texts):
        ax_bell.text(region_x_positions[i], max_y_for_text, txt, ha='center', fontsize=8, color='black',alpha=0.7)

    if logo2_resized:
        fig_page4.figimage(logo2_resized, fig_page4.bbox.xmax - 50 - logo2_resized.width, fig_page4.bbox.ymin + 30, zorder=3)

    pdf.savefig(fig_page4)
    plt.close(fig_page4)

    pdf.close()
    print(f"Report content generated: {output_pdf}")

    # --- بخش ادغام PDF و دانلود بدون تغییر ---
    writer = PdfWriter()
    merged_successfully = True
    # ... (کد ادغام و دانلود همانند قبل) ...
    try:
        if front_cover_path:
            try:
                front_pdf_reader = PdfReader(front_cover_path)
                if front_pdf_reader.pages:
                    writer.add_page(front_pdf_reader.pages[0])
                else: print(f"Warning: Front cover '{front_cover_path}' is empty.")
            except FileNotFoundError:
                print(f"Warning: Front cover '{front_cover_path}' not found. Skipping.")
                merged_successfully = False
            except Exception as e:
                print(f"Error reading front cover '{front_cover_path}': {e}. Skipping.")
                merged_successfully = False

        report_pdf_reader = PdfReader(output_pdf)
        for page in report_pdf_reader.pages:
            writer.add_page(page)

        if back_cover_path:
            try:
                back_pdf_reader = PdfReader(back_cover_path)
                if back_pdf_reader.pages:
                    writer.add_page(back_pdf_reader.pages[0])
                else: print(f"Warning: Back cover '{back_cover_path}' is empty.")
            except FileNotFoundError:
                print(f"Warning: Back cover '{back_cover_path}' not found. Skipping.")
                merged_successfully = False
            except Exception as e:
                print(f"Error reading back cover '{back_cover_path}': {e}. Skipping.")
                merged_successfully = False

        with open(output_pdf, 'wb') as f:
            writer.write(f)
        print(f"Merged PDF saved as: {output_pdf}")

    except Exception as e:
        print(f"An error occurred during PDF merging: {e}. The report '{output_pdf}' contains only the generated pages without covers potentially.")
        merged_successfully = False

    if merged_successfully:
        try:
            files.download(output_pdf)
            print(f"'{output_pdf}' download initiated.")
        except NameError: # Handles cases where 'files' is not defined (e.g. not in Colab)
            print("Skipping download: Not in Google Colab environment or 'files' object not available.")
        except Exception as e:
            print(f"An error occurred during download: {e}")
    else:
        print(f"Download of '{output_pdf}' skipped due to issues with cover pages or merging.")


# --- Example Usage with data from CSV ---
# این بخش بدون تغییر باقی می‌ماند
file_path = 'normalized_results_with_final_and_average.xlsx - Sheet1.csv'
data_for_report_df = pd.DataFrame()

labels_page1_cognitive_global = [
    'Visual Memory', 'Working Memory', 'Short-term Memory', 'Focus Attention', 'Updating',
    'Hand-Eye Coordination', 'Spatial Perception', 'Reaction Time', 'Processing Speed',
    'Planning and Reasoning'
]
labels_page2_assessments_global = [
    'Psychomotor Vigilance Test', 'Eye-Hand Coordination Test',
    'Eye-Hand Coordination MUD', 'Maze Test', 'Digit Span Test', 'Visual Memory Test'
]

try:
    full_df = pd.read_csv(file_path)
    print(f"Successfully loaded CSV: {file_path}")
    potential_index_col = full_df.columns[0]
    full_df[potential_index_col] = full_df[potential_index_col].astype(str)

    final_score_df = full_df[full_df[potential_index_col].str.lower() == 'final score']
    average_df = full_df[full_df[potential_index_col].str.lower() == 'average']

    if not final_score_df.empty:
        data_for_report_df = final_score_df.head(1)
        print("Found and using 'Final Score' row.")
    elif not average_df.empty:
        data_for_report_df = average_df.head(1)
        print("Found and using 'Average' row.")
    elif not full_df.empty:
        data_for_report_df = full_df.head(1)
        print(f"Warning: Neither 'Final Score' nor 'Average' found in column '{potential_index_col}'. Using the first data row.")
    else:
        print("Error: CSV file is empty or no specific summary row found. Using dummy data.")
        all_expected_labels = (labels_page1_cognitive_global + labels_page2_assessments_global)
        dummy_data_dict = {label: [np.random.rand()] for label in all_expected_labels}
        data_for_report_df = pd.DataFrame(dummy_data_dict)


    if data_for_report_df.empty :
        print("Critical Error: No data available for report. Using dummy data.")
        all_expected_labels = (labels_page1_cognitive_global + labels_page2_assessments_global)
        dummy_data_dict = {label: [np.random.rand()] for label in all_expected_labels}
        data_for_report_df = pd.DataFrame(dummy_data_dict)

    plot_maker(data_for_report_df)

except FileNotFoundError:
    print(f"Error: The file '{file_path}' was not found.")
    all_expected_labels = (labels_page1_cognitive_global + labels_page2_assessments_global)
    dummy_data_dict = {label: [np.random.rand()] for label in all_expected_labels}
    data_for_report_df = pd.DataFrame(dummy_data_dict)
    print("Generating report with dummy data for structure testing.")
    plot_maker(data_for_report_df)
except Exception as e:
    print(f"An error occurred: {e}")
    if data_for_report_df.empty:
        print("Creating dummy data due to an error during data loading/processing.")
        all_expected_labels = (labels_page1_cognitive_global + labels_page2_assessments_global)
        dummy_data_dict = {label: [np.random.rand()] for label in all_expected_labels}
        data_for_report_df = pd.DataFrame(dummy_data_dict)
        plot_maker(data_for_report_df)

Error: The file 'normalized_results_with_final_and_average.xlsx - Sheet1.csv' was not found.
Generating report with dummy data for structure testing.
Logo1 '/content/Logo1.png' loaded and resized to (120, 52).
Logo2 '/content/WhiteTree.png' loaded and resized.


Report content generated: report.pdf
Merged PDF saved as: report.pdf


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

'report.pdf' download initiated.
